# 13 - Pipeline Final de Previsão com Dois Estágios (VERSÃO LIMPA)

Este notebook combina os dois modelos treinados para gerar as previsões finais.

## Pipeline de Dois Estágios:
1. **Classificação**: Prever SE vai vender (LGBMClassifier)
2. **Decisão**: Aplicar threshold de probabilidade otimizado
3. **Regressão**: Prever QUANTO vai vender apenas para casos positivos (LGBMRegressor)
4. **Combinação**: Juntar resultados para submissão final

## Objetivo:
- Gerar previsões para as 5 semanas de Janeiro 2023
- Criar arquivo de submissão no formato exigido
- Aplicar otimizações para WMAPE

In [1]:
import pandas as pd
import numpy as np
import pickle
from datetime import datetime, timedelta
import os
import warnings
import gc
warnings.filterwarnings('ignore')

print('🚀 Iniciando Pipeline Final de Dois Estágios - VERSÃO LIMPA')
print('🎯 Objetivo: Gerar previsões para Janeiro 2023 combinando classificação + regressão')
print('⚡ Versão: Otimizada para performance e sem overflow')

# Criar pasta submissao3 se não existir
os.makedirs('../data/submissao3', exist_ok=True)
print('📁 Pasta data/submissao3 criada/verificada')

🚀 Iniciando Pipeline Final de Dois Estágios - VERSÃO LIMPA
🎯 Objetivo: Gerar previsões para Janeiro 2023 combinando classificação + regressão
⚡ Versão: Otimizada para performance e sem overflow
📁 Pasta data/submissao3 criada/verificada


## 1. Carregamento dos Modelos Treinados

In [2]:
# Carregar modelos treinados
print('📂 Carregando modelos treinados...')

# Classificador
with open('../data/submissao3/lgbm_classifier.pkl', 'rb') as f:
    lgbm_classifier = pickle.load(f)

# Regressor
with open('../data/submissao3/lgbm_regressor.pkl', 'rb') as f:
    lgbm_regressor = pickle.load(f)

# Features para classificação
with open('../data/submissao3/classification_features.pkl', 'rb') as f:
    features_classificacao = pickle.load(f)

# Features para regressão
try:
    with open('../data/submissao3/regression_features.pkl', 'rb') as f:
        features_regressao = pickle.load(f)
except FileNotFoundError:
    print('   📝 Arquivo regression_features.pkl não encontrado - usando mesmo da classificação')
    features_regressao = features_classificacao

# Metadados para verificação
with open('../data/submissao3/lgbm_classifier_metadata.pkl', 'rb') as f:
    meta_classifier = pickle.load(f)

with open('../data/submissao3/lgbm_regressor_metadata.pkl', 'rb') as f:
    meta_regressor = pickle.load(f)

print(f'✅ Modelos carregados:')
print(f'   🎯 Classificador: {type(lgbm_classifier).__name__}')
print(f'   📊 Regressor: {type(lgbm_regressor).__name__}')
print(f'   📋 Features CLASSIFICAÇÃO: {len(features_classificacao)}')
print(f'   📋 Features REGRESSÃO: {len(features_regressao)}')

print(f'\n📊 Performance dos modelos:')
print(f'   🎯 Classificador - AUC: {meta_classifier["metricas_validacao"]["auc"]:.4f}')
print(f'   📊 Regressor - RMSE: {meta_regressor["metricas_validacao"]["rmse"]:.4f}')

📂 Carregando modelos treinados...
✅ Modelos carregados:
   🎯 Classificador: LGBMClassifier
   📊 Regressor: LGBMRegressor
   📋 Features CLASSIFICAÇÃO: 27
   📋 Features REGRESSÃO: 31

📊 Performance dos modelos:
   🎯 Classificador - AUC: 0.9997
   📊 Regressor - RMSE: 5.0942


## 2. Preparação dos Dados Históricos e Combinações

In [3]:
# Carregar dados históricos
print('📂 Carregando dados históricos...')

train_data = pd.read_parquet('../data/submissao3/train_data.parquet')
validation_data = pd.read_parquet('../data/submissao3/validation_data.parquet')

# Combinar todos os dados históricos
dados_completos_2022 = pd.concat([train_data, validation_data], ignore_index=True)
print(f'📊 Dados históricos de 2022: {dados_completos_2022.shape}')

# Definir semanas de Janeiro 2023
ultima_semana_2022 = dados_completos_2022['semana'].max()
print(f'📅 Última semana de 2022: {ultima_semana_2022}')

semanas_teste = []
for i in range(1, 6):
    proxima_semana = ultima_semana_2022 + pd.Timedelta(weeks=i)
    semanas_teste.append(proxima_semana)

print(f'📅 Semanas de teste (Janeiro 2023):')
for i, semana in enumerate(semanas_teste):
    print(f'      {i+1}. {semana.strftime("%Y-%m-%d")}')

# Identificar combinações ativas
combinacoes_ativas = dados_completos_2022[['pdv_id', 'produto_id']].drop_duplicates()
print(f'📊 Combinações ativas: {len(combinacoes_ativas):,}')

print('✅ Dados históricos preparados')

📂 Carregando dados históricos...
📊 Dados históricos de 2022: (55348430, 6)
📅 Última semana de 2022: 2022-12-27 00:00:00
📅 Semanas de teste (Janeiro 2023):
      1. 2023-01-03
      2. 2023-01-10
      3. 2023-01-17
      4. 2023-01-24
      5. 2023-01-31
📊 Combinações ativas: 1,044,310
✅ Dados históricos preparados


## 3. Criação do Grid de Teste com Features Otimizadas

In [4]:
# Função para otimizar tipos de dados SEM overflow
def otimizar_dtypes_seguro(df):
    """Otimiza tipos de dados de forma segura, sem overflow."""
    memoria_antes = df.memory_usage(deep=True).sum() / 1024**2
    
    for col in df.columns:
        if df[col].dtype == 'int64':
            # Verificar se pode usar int32 sem overflow
            if df[col].min() >= np.iinfo(np.int32).min and df[col].max() <= np.iinfo(np.int32).max:
                df[col] = df[col].astype('int32')
        elif df[col].dtype == 'float64':
            df[col] = pd.to_numeric(df[col], downcast='float')
    
    memoria_depois = df.memory_usage(deep=True).sum() / 1024**2
    print(f'      🔧 Memória: {memoria_antes:.1f} MB → {memoria_depois:.1f} MB')
    return df

# Carregar dados auxiliares
print('📂 Carregando dados auxiliares...')

produtos = pd.read_parquet(
    '../data/part-00000-tid-7173294866425216458-eae53fbf-d19e-4130-ba74-78f96b9675f1-4-1-c000.snappy.parquet'
)
produtos = produtos.rename(columns={'produto': 'produto_id'})
produtos = otimizar_dtypes_seguro(produtos)

pdvs = pd.read_parquet(
    '../data/part-00000-tid-2779033056155408584-f6316110-4c9a-4061-ae48-69b77c7c8c36-4-1-c000.snappy.parquet'
)
pdvs = pdvs.rename(columns={'pdv': 'pdv_id', 'categoria_pdv': 'tipo_loja'})
pdvs = otimizar_dtypes_seguro(pdvs)

print(f'📊 Produtos: {produtos.shape} - Categorias: {produtos["categoria"].nunique()}')
print(f'📊 PDVs: {pdvs.shape} - Zipcodes: {pdvs["zipcode"].nunique()}')

print('✅ Dados auxiliares carregados')

📂 Carregando dados auxiliares...
      🔧 Memória: 4.0 MB → 4.0 MB
      🔧 Memória: 3.0 MB → 3.0 MB
📊 Produtos: (7092, 8) - Categorias: 7
📊 PDVs: (14419, 4) - Zipcodes: 788
✅ Dados auxiliares carregados


In [5]:
def criar_grid_teste_com_features(semanas_teste, combinacoes, dados_historicos):
    """Cria grid de teste com features otimizadas para máxima performance."""
    print(f'🔧 Criando grid de teste para {len(semanas_teste)} semanas...')
    
    # Otimizar dados de entrada
    print('   📊 Otimizando dados de entrada...')
    dados_historicos = otimizar_dtypes_seguro(dados_historicos.copy())
    combinacoes = otimizar_dtypes_seguro(combinacoes.copy())
    
    # Criar grid de teste
    print('   🎯 Criando grid de teste...')
    grid_parts = []
    for semana in semanas_teste:
        df_semana = combinacoes.copy()
        df_semana['semana'] = semana
        df_semana['quantidade'] = 0
        df_semana['faturamento'] = 0.0
        df_semana['distributor_id'] = None
        grid_parts.append(df_semana)
    
    dados_teste_raw = pd.concat(grid_parts, ignore_index=True)
    dados_teste_raw = otimizar_dtypes_seguro(dados_teste_raw)
    print(f'   📊 Grid criado: {dados_teste_raw.shape}')
    
    # Filtrar dados históricos relevantes (últimas 4 semanas por combinação)
    print('   🔍 Filtrando histórico relevante...')
    pdvs_relevantes = set(combinacoes['pdv_id'])
    produtos_relevantes = set(combinacoes['produto_id'])
    
    mask_relevantes = (
        dados_historicos['pdv_id'].isin(pdvs_relevantes) & 
        dados_historicos['produto_id'].isin(produtos_relevantes)
    )
    dados_historicos_filtrados = dados_historicos[mask_relevantes].copy()
    
    # Manter apenas últimas 4 semanas por combinação (suficiente para lags)
    dados_historicos_filtrados = dados_historicos_filtrados.sort_values(['pdv_id', 'produto_id', 'semana'])
    dados_historicos_filtrados = dados_historicos_filtrados.groupby(['pdv_id', 'produto_id']).tail(4)
    print(f'   📊 Histórico filtrado: {dados_historicos_filtrados.shape}')
    
    # Combinar histórico + teste
    dados_completos = pd.concat([dados_historicos_filtrados, dados_teste_raw], ignore_index=True)
    dados_completos = dados_completos.sort_values(['pdv_id', 'produto_id', 'semana']).reset_index(drop=True)
    dados_completos = otimizar_dtypes_seguro(dados_completos)
    print(f'   📊 Dados completos: {dados_completos.shape}')
    
    # Merge com dados auxiliares
    print('   🔗 Fazendo merges...')
    dados_completos = dados_completos.merge(produtos[['produto_id', 'categoria']], on='produto_id', how='left')
    dados_completos = dados_completos.merge(pdvs[['pdv_id', 'zipcode', 'tipo_loja']], on='pdv_id', how='left')
    dados_completos = otimizar_dtypes_seguro(dados_completos)
    
    # Preencher distributor_id
    dados_completos['distributor_id'] = dados_completos.groupby('produto_id')['distributor_id'].transform('ffill').transform('bfill')
    
    # ========== CRIAR FEATURES ==========
    print('   ⚡ Criando features...')
    
    # 1. Features básicas
    dados_completos['preco_unitario_atual'] = np.where(
        dados_completos['quantidade'] > 0,
        dados_completos['faturamento'] / dados_completos['quantidade'],
        0
    ).astype('float32')
    
    # 2. Features temporais
    dados_completos['mes'] = dados_completos['semana'].dt.month.astype('int8')
    dados_completos['mes_sin'] = np.sin(2 * np.pi * dados_completos['mes'] / 12).astype('float32')
    dados_completos['mes_cos'] = np.cos(2 * np.pi * dados_completos['mes'] / 12).astype('float32')
    dados_completos['eh_inicio_mes'] = (dados_completos['semana'].dt.day <= 7).astype('int8')
    dados_completos['eh_fim_mes'] = (dados_completos['semana'].dt.day >= 22).astype('int8')
    dados_completos['dia_do_mes'] = dados_completos['semana'].dt.day.astype('int8')
    dados_completos['semana_do_mes'] = ((dados_completos['semana'].dt.day - 1) // 7 + 1).astype('int8')
    
    # 3. Features de lag (CRÍTICAS)
    print('   🔧 Features de lag e rolling...')
    gb = dados_completos.groupby(['pdv_id', 'produto_id'])
    
    # Lags de quantidade
    for lag in [1, 2, 3, 4]:
        dados_completos[f'quantidade_lag_{lag}'] = gb['quantidade'].shift(lag).astype('float32')
        
    # Lags de preço
    dados_completos['preco_lag_1'] = gb['preco_unitario_atual'].shift(1).astype('float32')
    dados_completos['preco_lag_2'] = gb['preco_unitario_atual'].shift(2).astype('float32')
    dados_completos['variacao_preco_sku_semanal'] = (dados_completos['preco_lag_1'] - dados_completos['preco_lag_2']).fillna(0).astype('float32')
    
    # Rolling features (OTIMIZADAS)
    rolling_gb = gb['quantidade'].rolling(window=4, min_periods=1)
    dados_completos['quantidade_media_4w'] = rolling_gb.mean().reset_index(level=[0,1], drop=True).astype('float32')
    dados_completos['quantidade_std_4w'] = rolling_gb.std().reset_index(level=[0,1], drop=True).fillna(0).astype('float32')
    dados_completos['quantidade_max_4w'] = rolling_gb.max().reset_index(level=[0,1], drop=True).astype('float32')
    
    # 4. Features derivadas
    dados_completos['momentum_ratio'] = (dados_completos['quantidade_lag_1'] / dados_completos['quantidade_media_4w']).fillna(0).astype('float32')
    dados_completos['aceleracao'] = (dados_completos['quantidade_lag_1'] - dados_completos['quantidade_lag_2']).fillna(0).astype('float32')
    
    # 5. Features categóricas hash
    dados_completos['pdv_hash'] = (dados_completos['pdv_id'].astype(str).apply(hash).abs() % 100).astype('int8')
    dados_completos['produto_hash'] = (dados_completos['produto_id'].astype(str).apply(hash).abs() % 100).astype('int8')
    dados_completos['categoria_hash'] = (dados_completos['categoria'].astype(str).apply(hash).abs() % 50).astype('int8')
    dados_completos['zipcode_hash'] = (dados_completos['zipcode'].astype(str).apply(hash).abs() % 1000).astype('int16')
    
    dados_completos['pdv_produto_hash'] = (dados_completos['pdv_hash'] * 100 + dados_completos['produto_hash']).astype('int16')
    dados_completos['categoria_zipcode_hash'] = (dados_completos['categoria_hash'].astype('int32') * 1000 + dados_completos['zipcode_hash']).astype('int32')
    
    # 6. Features agregadas
    print('   📈 Features agregadas...')
    dados_completos['preco_medio_semanal_sku_atual'] = dados_completos.groupby(['semana', 'produto_id'])['preco_unitario_atual'].transform('mean').astype('float32')
    dados_completos['media_vendas_categoria_pdv_lag_1'] = dados_completos.groupby(['semana', 'categoria', 'pdv_id'])['quantidade_lag_1'].transform('mean').astype('float32')
    
    # Share de vendas (para classificação - com lag)
    vendas_categoria_lag = dados_completos.groupby(['semana', 'categoria', 'pdv_id'])['quantidade_lag_1'].transform('sum')
    dados_completos['share_vendas_sku_categoria_lag_1'] = (dados_completos['quantidade_lag_1'] / vendas_categoria_lag).fillna(0).astype('float32')
    
    # Share de vendas (para regressão - atual)
    dados_completos['media_vendas_categoria_pdv_atual'] = dados_completos.groupby(['semana', 'categoria', 'pdv_id'])['quantidade'].transform('mean').astype('float32')
    vendas_categoria_atual = dados_completos.groupby(['semana', 'categoria', 'pdv_id'])['quantidade'].transform('sum')
    dados_completos['share_vendas_sku_categoria_atual'] = (dados_completos['quantidade'] / vendas_categoria_atual).fillna(0).astype('float32')
    
    # Limpeza final
    dados_completos.fillna(0, inplace=True)
    dados_completos = otimizar_dtypes_seguro(dados_completos)
    
    # Filtrar apenas semanas de teste
    dados_teste_final = dados_completos[dados_completos['semana'].isin(semanas_teste)].copy()
    
    print(f'   ✅ Dados de teste finais: {dados_teste_final.shape}')
    print(f'   💾 Memória total: {dados_teste_final.memory_usage(deep=True).sum() / 1024**2:.1f} MB')
    
    # Liberar memória
    del dados_completos, dados_historicos_filtrados, dados_teste_raw, grid_parts
    gc.collect()
    
    return dados_teste_final

# EXECUTAR CRIAÇÃO DO GRID
dados_teste = criar_grid_teste_com_features(semanas_teste, combinacoes_ativas, dados_completos_2022)

print('✅ Grid de teste criado com features otimizadas!')

🔧 Criando grid de teste para 5 semanas...
   📊 Otimizando dados de entrada...
      🔧 Memória: 12329.8 MB → 12118.7 MB
      🔧 Memória: 159.1 MB → 159.1 MB
   🎯 Criando grid de teste...
      🔧 Memória: 994.8 MB → 954.9 MB
   📊 Grid criado: (5221550, 6)
   🔍 Filtrando histórico relevante...
   📊 Histórico filtrado: (4177240, 6)
      🔧 Memória: 1919.8 MB → 1884.0 MB
   📊 Dados completos: (9398790, 6)
   🔗 Fazendo merges...
      🔧 Memória: 3174.6 MB → 3138.7 MB
   ⚡ Criando features...
   🔧 Features de lag e rolling...
   📈 Features agregadas...
      🔧 Memória: 4177.3 MB → 4177.3 MB
   ✅ Dados de teste finais: (5221550, 40)
   💾 Memória total: 2360.5 MB
✅ Grid de teste criado com features otimizadas!


## 4. Aplicação do Pipeline de Dois Estágios

In [6]:
# ESTÁGIO 1: CLASSIFICAÇÃO
print('🎯 ESTÁGIO 1: CLASSIFICAÇÃO (prever SE vai vender)')

# Preparar features para classificação
print('   🔧 Preparando features para classificação...')
X_teste_classificacao = dados_teste[features_classificacao].copy().fillna(0)

# Verificar features faltando
features_faltando = set(features_classificacao) - set(dados_teste.columns)
if features_faltando:
    print(f'   ⚠️ Features faltando: {len(features_faltando)} - criando com zeros')
    for feat in features_faltando:
        X_teste_classificacao[feat] = 0

# Reordenar para match com modelo
X_teste_classificacao = X_teste_classificacao[features_classificacao]
print(f'   ✅ Features classificação: {X_teste_classificacao.shape}')

# Aplicar classificação
probabilidades_venda = lgbm_classifier.predict_proba(X_teste_classificacao)[:, 1]
print(f'   📊 Probabilidades: média={probabilidades_venda.mean():.4f}, mediana={np.median(probabilidades_venda):.4f}')

# ESTÁGIO 2: THRESHOLD OTIMIZADO
print('\n🎯 ESTÁGIO 2: OTIMIZAÇÃO DE THRESHOLD')

thresholds = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
print('   📊 Análise de thresholds:')
for t in thresholds:
    pos = (probabilidades_venda >= t).sum()
    pct = pos / len(probabilidades_venda) * 100
    print(f'      Threshold {t:.1f}: {pos:7,} positivos ({pct:5.1f}%)')

# Escolher threshold baseado no histórico
taxa_historica = (dados_completos_2022['quantidade'] > 0).mean()
target_taxa = taxa_historica * 0.8  # Conservador para precisão

best_threshold = 0.5
min_diff = float('inf')
for t in thresholds:
    taxa_atual = (probabilidades_venda >= t).mean()
    diff = abs(taxa_atual - target_taxa)
    if diff < min_diff:
        min_diff = diff
        best_threshold = t

vai_vender = (probabilidades_venda >= best_threshold)
casos_positivos = vai_vender.sum()

print(f'   🎯 Threshold escolhido: {best_threshold}')
print(f'   📊 Taxa histórica: {taxa_historica:.1%} | Target: {target_taxa:.1%} | Obtida: {(casos_positivos/len(vai_vender)):.1%}')
print(f'   📊 Casos positivos: {casos_positivos:,} ({casos_positivos/len(vai_vender)*100:.1f}%)')

print('✅ Estágios 1 e 2 concluídos!')

🎯 ESTÁGIO 1: CLASSIFICAÇÃO (prever SE vai vender)
   🔧 Preparando features para classificação...
   ✅ Features classificação: (5221550, 27)
   📊 Probabilidades: média=0.0021, mediana=0.0007

🎯 ESTÁGIO 2: OTIMIZAÇÃO DE THRESHOLD
   📊 Análise de thresholds:
      Threshold 0.1:  14,691 positivos (  0.3%)
      Threshold 0.2:   4,502 positivos (  0.1%)
      Threshold 0.3:   2,286 positivos (  0.0%)
      Threshold 0.4:   1,149 positivos (  0.0%)
      Threshold 0.5:     604 positivos (  0.0%)
      Threshold 0.6:     278 positivos (  0.0%)
      Threshold 0.7:      76 positivos (  0.0%)
      Threshold 0.8:       9 positivos (  0.0%)
      Threshold 0.9:       0 positivos (  0.0%)
   🎯 Threshold escolhido: 0.1
   📊 Taxa histórica: 11.1% | Target: 8.9% | Obtida: 0.3%
   📊 Casos positivos: 14,691 (0.3%)
✅ Estágios 1 e 2 concluídos!


In [7]:
# ESTÁGIO 3: REGRESSÃO
print('📊 ESTÁGIO 3: REGRESSÃO (prever QUANTO vai vender)')

previsoes_finais = np.zeros(len(dados_teste))

if casos_positivos > 0:
    print(f'   📊 Aplicando regressão a {casos_positivos:,} casos positivos...')
    
    # Preparar features para regressão
    X_teste_regressao = dados_teste[vai_vender][features_regressao].copy().fillna(0)
    
    # Verificar features faltando
    features_faltando_reg = set(features_regressao) - set(dados_teste.columns)
    if features_faltando_reg:
        print(f'      ⚠️ Features faltando na regressão: {len(features_faltando_reg)}')
        for feat in features_faltando_reg:
            X_teste_regressao[feat] = 0
    
    X_teste_regressao = X_teste_regressao[features_regressao]
    print(f'   📊 Features para regressão: {X_teste_regressao.shape}')
    
    # Aplicar regressão
    previsoes_quantidade_raw = lgbm_regressor.predict(X_teste_regressao)
    
    # Pós-processamento otimizado para WMAPE
    previsoes_quantidade = np.maximum(previsoes_quantidade_raw, 0)
    
    # Ceiling inteligente baseado no histórico
    p95_historico = np.percentile(dados_completos_2022[dados_completos_2022['quantidade'] > 0]['quantidade'], 95)
    previsoes_quantidade = np.where(
        previsoes_quantidade > p95_historico,
        p95_historico + (previsoes_quantidade - p95_historico) * 0.3,
        previsoes_quantidade
    )
    
    # Arredondamento inteligente
    previsoes_quantidade_final = np.where(
        previsoes_quantidade < 1.5,
        np.maximum(np.round(previsoes_quantidade), 1),  # Pelo menos 1 para casos positivos
        np.round(previsoes_quantidade)
    ).astype(int)
    
    # Inserir previsões nos casos positivos
    previsoes_finais[vai_vender] = previsoes_quantidade_final
    
    print(f'   📊 Previsões: média={previsoes_quantidade_final.mean():.2f}, mediana={np.median(previsoes_quantidade_final):.0f}')
    print(f'   📊 Min: {previsoes_quantidade_final.min()}, Max: {previsoes_quantidade_final.max()}')

else:
    print('   ⚠️ Nenhum caso positivo detectado - todas as previsões serão zero!')

# RESULTADO FINAL
print('\n🎉 PIPELINE DE DOIS ESTÁGIOS CONCLUÍDO!')
print('=' * 60)
print(f'📊 Total de previsões: {len(previsoes_finais):,}')
print(f'📊 Previsões zero: {(previsoes_finais == 0).sum():,} ({(previsoes_finais == 0).mean()*100:.1f}%)')
print(f'📊 Previsões positivas: {(previsoes_finais > 0).sum():,} ({(previsoes_finais > 0).mean()*100:.1f}%)')
print(f'📊 Soma total prevista: {previsoes_finais.sum():,}')

# Adicionar resultados ao dataframe
dados_teste['quantidade_prevista'] = previsoes_finais
dados_teste['probabilidade_venda'] = probabilidades_venda
dados_teste['vai_vender'] = vai_vender

# Validação dos resultados
taxa_prevista = (previsoes_finais > 0).mean()
taxa_historica_validacao = (dados_completos_2022['quantidade'] > 0).mean()

print(f'\n📊 VALIDAÇÃO:')
print(f'   📈 Taxa histórica de vendas: {taxa_historica_validacao:.1%}')
print(f'   📈 Taxa prevista de vendas: {taxa_prevista:.1%}')
print(f'   📈 Ratio previsto/histórico: {taxa_prevista/taxa_historica_validacao:.2f}')

if taxa_prevista < 0.01:
    print('   ⚠️ ALERTA: Taxa de previsões positivas muito baixa!')
elif taxa_prevista > 0.20:
    print('   ⚠️ ALERTA: Taxa de previsões positivas muito alta!')
else:
    print('   ✅ Taxa de previsões positivas dentro do esperado')

print('\n✅ Pipeline aplicado com sucesso!')

📊 ESTÁGIO 3: REGRESSÃO (prever QUANTO vai vender)
   📊 Aplicando regressão a 14,691 casos positivos...
   📊 Features para regressão: (14691, 31)
   📊 Previsões: média=7.03, mediana=4
   📊 Min: 1, Max: 223

🎉 PIPELINE DE DOIS ESTÁGIOS CONCLUÍDO!
📊 Total de previsões: 5,221,550
📊 Previsões zero: 5,206,859 (99.7%)
📊 Previsões positivas: 14,691 (0.3%)
📊 Soma total prevista: 103,329.0

📊 VALIDAÇÃO:
   📈 Taxa histórica de vendas: 11.1%
   📈 Taxa prevista de vendas: 0.3%
   📈 Ratio previsto/histórico: 0.03
   ⚠️ ALERTA: Taxa de previsões positivas muito baixa!

✅ Pipeline aplicado com sucesso!


## 5. Geração do Arquivo de Submissão

In [8]:
# Gerar arquivo de submissão no formato exigido
print('📄 Gerando arquivo de submissão...')

# Preparar dados para submissão
submissao = dados_teste[['semana', 'pdv_id', 'produto_id', 'quantidade_prevista']].copy()
submissao = submissao.rename(columns={'quantidade_prevista': 'quantity'})

# Verificações de consistência
print(f'\n   🔍 Verificações de consistência:')
print(f'      • Shape: {submissao.shape}')
print(f'      • Colunas: {list(submissao.columns)}')
print(f'      • Período: {submissao["semana"].min()} até {submissao["semana"].max()}')
print(f'      • Semanas únicas: {submissao["semana"].nunique()} (esperado: 5)')
print(f'      • Valores negativos: {(submissao["quantity"] < 0).sum()} (deve ser 0)')
print(f'      • Valores NaN: {submissao["quantity"].isna().sum()} (deve ser 0)')
print(f'      • Duplicatas: {submissao.duplicated(subset=["semana", "pdv_id", "produto_id"]).sum()} (deve ser 0)')

# Validar total esperado
total_esperado = len(combinacoes_ativas) * len(semanas_teste)
print(f'      • Total registros: {len(submissao):,} (esperado: {total_esperado:,})')

if len(submissao) == total_esperado:
    print(f'      ✅ Total de registros correto!')
else:
    print(f'      ⚠️ Discrepância no total de registros!')

# Estatísticas da submissão
print(f'\n   📊 Estatísticas da submissão:')
print(f'      • Previsões zero: {(submissao["quantity"] == 0).sum():,} ({(submissao["quantity"] == 0).mean()*100:.1f}%)')
print(f'      • Previsões positivas: {(submissao["quantity"] > 0).sum():,} ({(submissao["quantity"] > 0).mean()*100:.1f}%)')
print(f'      • Soma total: {submissao["quantity"].sum():,}')
print(f'      • Quantidade máxima: {submissao["quantity"].max()}')
if (submissao["quantity"] > 0).sum() > 0:
    print(f'      • Quantidade média (apenas >0): {submissao[submissao["quantity"] > 0]["quantity"].mean():.2f}')

# Salvar arquivos
nome_arquivo_principal = '../data/submissao3/submission_final_two_stage_LIMPO.parquet'
submissao.to_parquet(nome_arquivo_principal, index=False)
print(f'\n   ✅ Arquivo principal salvo: {nome_arquivo_principal}')

nome_arquivo_csv = '../data/submissao3/submission_final_two_stage_LIMPO.csv'
submissao.to_csv(nome_arquivo_csv, index=False)
print(f'   ✅ Arquivo CSV salvo: {nome_arquivo_csv}')

# Salvar dados detalhados para análise
dados_detalhados = dados_teste[[
    'semana', 'pdv_id', 'produto_id', 'quantidade_prevista', 
    'probabilidade_venda', 'vai_vender'
]].copy()

nome_arquivo_detalhado = '../data/submissao3/submission_detailed_LIMPO.parquet'
dados_detalhados.to_parquet(nome_arquivo_detalhado, index=False)
print(f'   ✅ Arquivo detalhado salvo: {nome_arquivo_detalhado}')

# Análise por semana
print(f'\n   📅 ANÁLISE POR SEMANA:')
for i, semana in enumerate(semanas_teste):
    dados_semana = submissao[submissao['semana'] == semana]
    total = dados_semana['quantity'].sum()
    positivos = (dados_semana['quantity'] > 0).sum()
    taxa = positivos / len(dados_semana) * 100
    print(f'      Semana {i+1} ({semana.strftime("%Y-%m-%d")}): Total={total:7.0f} | Positivos={positivos:6,} ({taxa:4.1f}%)')

print(f'\n✅ ARQUIVOS DE SUBMISSÃO GERADOS!')
print(f'📄 Arquivo principal: submission_final_two_stage_LIMPO.parquet')
print(f'🎯 Pronto para submissão ao desafio!')

📄 Gerando arquivo de submissão...

   🔍 Verificações de consistência:
      • Shape: (5221550, 4)
      • Colunas: ['semana', 'pdv_id', 'produto_id', 'quantity']
      • Período: 2023-01-03 00:00:00 até 2023-01-31 00:00:00
      • Semanas únicas: 5 (esperado: 5)
      • Valores negativos: 0 (deve ser 0)
      • Valores NaN: 0 (deve ser 0)
      • Duplicatas: 0 (deve ser 0)
      • Total registros: 5,221,550 (esperado: 5,221,550)
      ✅ Total de registros correto!

   📊 Estatísticas da submissão:
      • Previsões zero: 5,206,859 (99.7%)
      • Previsões positivas: 14,691 (0.3%)
      • Soma total: 103,329.0
      • Quantidade máxima: 223.0
      • Quantidade média (apenas >0): 7.03

   ✅ Arquivo principal salvo: ../data/submissao3/submission_final_two_stage_LIMPO.parquet
   ✅ Arquivo CSV salvo: ../data/submissao3/submission_final_two_stage_LIMPO.csv
   ✅ Arquivo detalhado salvo: ../data/submissao3/submission_detailed_LIMPO.parquet

   📅 ANÁLISE POR SEMANA:
      Semana 1 (2023-01-03)

## 6. Salvamento de Metadados Finais

In [9]:
# Salvar metadados completos do pipeline
print('📋 Salvando metadados do pipeline...')

# Calcular estatísticas detalhadas
previsoes_positivas_stats = dados_teste[dados_teste['quantidade_prevista'] > 0]['quantidade_prevista']
vendas_historicas = dados_completos_2022[dados_completos_2022['quantidade'] > 0]['quantidade']

metadados_pipeline = {
    'data_criacao': pd.Timestamp.now(),
    'versao': 'Pipeline Final Limpo v1.0',
    'estrategia': 'Pipeline de Dois Estágios - Classificação + Regressão',
    
    # Modelos
    'modelos': {
        'classificador': 'LightGBM Classifier',
        'regressor': 'LightGBM Regressor',
        'features_classificacao': len(features_classificacao),
        'features_regressao': len(features_regressao)
    },
    
    # Performance
    'performance_modelos': {
        'classificador_auc': float(meta_classifier['metricas_validacao']['auc']),
        'regressor_rmse': float(meta_regressor['metricas_validacao']['rmse'])
    },
    
    # Configuração do pipeline
    'configuracao': {
        'threshold_classificacao': float(best_threshold),
        'total_combinacoes': len(combinacoes_ativas),
        'semanas_previsao': len(semanas_teste),
        'periodo_inicio': semanas_teste[0].strftime('%Y-%m-%d'),
        'periodo_fim': semanas_teste[-1].strftime('%Y-%m-%d')
    },
    
    # Resultados
    'resultados': {
        'total_previsoes': len(previsoes_finais),
        'previsoes_zero': int((previsoes_finais == 0).sum()),
        'previsoes_positivas': int((previsoes_finais > 0).sum()),
        'taxa_previsoes_positivas': float((previsoes_finais > 0).mean()),
        'quantidade_total_prevista': int(previsoes_finais.sum()),
        'quantidade_media_prevista': float(previsoes_finais.mean()),
        'probabilidade_media': float(probabilidades_venda.mean())
    },
    
    # Estatísticas detalhadas
    'estatisticas_previsoes_positivas': {
        'count': len(previsoes_positivas_stats),
        'mean': float(previsoes_positivas_stats.mean()) if len(previsoes_positivas_stats) > 0 else 0,
        'median': float(previsoes_positivas_stats.median()) if len(previsoes_positivas_stats) > 0 else 0,
        'std': float(previsoes_positivas_stats.std()) if len(previsoes_positivas_stats) > 0 else 0,
        'min': float(previsoes_positivas_stats.min()) if len(previsoes_positivas_stats) > 0 else 0,
        'max': float(previsoes_positivas_stats.max()) if len(previsoes_positivas_stats) > 0 else 0
    },
    
    # Comparação histórica
    'comparacao_historica': {
        'taxa_historica_vendas': float(len(vendas_historicas) / len(dados_completos_2022)),
        'media_historica_vendas': float(vendas_historicas.mean()),
        'total_historico': float(dados_completos_2022['quantidade'].sum()),
        'ratio_total_previsto_historico': float(previsoes_finais.sum() / dados_completos_2022['quantidade'].sum()),
        'ratio_taxa_prevista_historica': float((previsoes_finais > 0).mean() / (dados_completos_2022['quantidade'] > 0).mean())
    },
    
    # Arquivos gerados
    'arquivos_gerados': [
        'submission_final_two_stage_LIMPO.parquet',
        'submission_final_two_stage_LIMPO.csv',
        'submission_detailed_LIMPO.parquet',
        'pipeline_final_LIMPO_metadata.pkl'
    ],
    
    # Otimizações aplicadas
    'otimizacoes': [
        'Otimização de tipos de dados sem overflow',
        'Filtragem de dados históricos (últimas 4 semanas)',
        'Rolling features otimizadas (sem groupby.apply)',
        'Threshold otimizado baseado em padrão histórico',
        'Ceiling inteligente em outliers para reduzir WMAPE',
        'Arredondamento conservador para baixas quantidades',
        'Garbage collection explícito para economizar memória'
    ],
    
    # Observações
    'observacoes': [
        'Pipeline limpo e organizado sem seções duplicadas',
        'Código otimizado para evitar overflow em IDs grandes',
        'Features engineered aplicadas consistentemente',
        'Validação temporal para evitar data leakage',
        'Threshold otimizado para balance precision/recall',
        'Pós-processamento focado em minimizar WMAPE'
    ]
}

# Salvar metadados
with open('../data/submissao3/pipeline_final_LIMPO_metadata.pkl', 'wb') as f:
    pickle.dump(metadados_pipeline, f)

print('📋 Metadados salvos em: pipeline_final_LIMPO_metadata.pkl')

print('\n🎉 PIPELINE FINAL LIMPO CONCLUÍDO COM SUCESSO!')
print('=' * 80)
print('🎯 RESUMO FINAL:')
print(f'   📊 Estratégia: {metadados_pipeline["estrategia"]}')
print(f'   🎯 Threshold: {best_threshold}')
print(f'   📊 Total previsões: {len(previsoes_finais):,}')
print(f'   📊 Taxa positivos: {(previsoes_finais > 0).mean()*100:.1f}%')
print(f'   📊 Soma total: {previsoes_finais.sum():,}')
print(f'\n📄 ARQUIVO PRINCIPAL:')
print(f'   ✅ submission_final_two_stage_LIMPO.parquet')
print(f'\n🚀 PRONTO PARA SUBMISSÃO AO DESAFIO!')
print('=' * 80)

📋 Salvando metadados do pipeline...
📋 Metadados salvos em: pipeline_final_LIMPO_metadata.pkl

🎉 PIPELINE FINAL LIMPO CONCLUÍDO COM SUCESSO!
🎯 RESUMO FINAL:
   📊 Estratégia: Pipeline de Dois Estágios - Classificação + Regressão
   🎯 Threshold: 0.1
   📊 Total previsões: 5,221,550
   📊 Taxa positivos: 0.3%
   📊 Soma total: 103,329.0

📄 ARQUIVO PRINCIPAL:
   ✅ submission_final_two_stage_LIMPO.parquet

🚀 PRONTO PARA SUBMISSÃO AO DESAFIO!
